In [3]:
import os
import json
import numpy as np
import tensorflow as tf
import tensorflow.compat.v1 as tf1

from tensorflow.compat.v1.keras.layers import Dense, LSTM,GRU,Bidirectional
from tensorflow.compat.v1.keras.layers import Activation, Dropout
#from keras.models import Sequential, load_model
from tensorflow.compat.v1.keras.models import Sequential, load_model



import pandas as pd

In [4]:
def vector(i, n):
    """
    Return vector of n "zeros" and a "one" at index i
    """
    tmp = np.zeros(n)
    tmp[i] = 1
    return tmp

In [5]:

def load_training(csv_file):

    data = pd.read_csv(csv_file)
    #print(data)
    print(data.head())
    print(data.shape)
    data= data.loc[data['percent']> 0.8]  #filtering names based on their prediction percentages
    print(data.shape)
    print(data.head())

    max_len = data.name.map(len).max()
    print("max_len" ,max_len)

    vocab = set(' '.join([str(i) for i in data['name']]))
    vocab.add('END')
    char_index = dict((c, i) for i, c in enumerate(vocab))
    vocab_len = len(vocab)
    print("vocab_len",vocab_len)

    msk = np.random.rand(len(data)) < 0.7
    data_train = data[msk]
    data_test = data[~msk]

    train_x = data_train['name']
    train_x = [list(i)+['END']*(max_len-len(i)) for i in train_x]
    train_x = [[vector(char_index[j], vocab_len) for j in i] for i in train_x]
    train_y = [([1, 0] if i == 'M' else [0, 1]) for i in data_train['gender']]
    train_x = np.asarray(train_x)
    train_y = np.asarray(train_y)
    print(train_x[0],train_y[0])

    print(train_x.shape,train_y.shape)
    test_x = data_test['name']
    test_x = [list(i)+['END']*(max_len-len(i)) for i in test_x]
    test_x = [[vector(char_index[j], vocab_len) for j in i] for i in test_x]
    test_y = [([1, 0] if i == 'M' else [0, 1]) for i in data_test['gender']]
    test_x = np.asarray(test_x)
    test_y = np.asarray(test_y)

    return train_x, train_y, test_x, test_y, int(max_len), int(vocab_len), char_index


In [6]:
train_x, train_y, test_x, test_y, max_len, vocab_len, char_index = load_training('E://NCS//NCS_Gender_Prediction_Kuruparan//data//name_gender.csv')



       name gender  percent
0    Abiyah      F      0.5
1    Adason      F      0.5
2  Adeyinka      F      0.5
3   Alexiel      F      0.5
4    Alique      F      0.5
(95025, 3)
(91890, 3)
       name gender   percent
3135  Emrys      M  0.800633
3136  Kasyn      M  0.800872
3137    Yer      F  0.801120
3138  Haile      F  0.801611
3139  Adean      F  0.801724
max_len 15
vocab_len 54
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 1. 0. 0. 0. 0.

In [7]:

def save_model(model, data, network_path):
    """
    Save model architecture and weights into files
    """
    if not os.path.exists(network_path):
        os.makedirs(network_path)
    open(os.path.join(network_path, 'data.json'), 'w').write(json.dumps(data))
    model.save(os.path.join(network_path, 'server.model'))


In [8]:

def read_model(network_path):
    """
    Load model architecture and weights from files
    """
    if not os.path.exists(network_path):
        raise ValueError('Path not found : {}'.format(network_path))
    data = json.loads(open(os.path.join(network_path, 'data.json')).read())
    model = load_model(os.path.join(network_path, 'server.model'))
    return model, data


In [9]:
# build model  LSTM

callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10)

model = Sequential()
model.add(LSTM(256, return_sequences=True, input_shape=(max_len, vocab_len)))
model.add(Dropout(0.2))
model.add(LSTM(512, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(2))
model.add(Activation('softmax'))
model.summary()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


# train model
model.fit(train_x, train_y, batch_size=128, epochs=200, validation_data=(test_x, test_y), callbacks=[callback])

# test model
loss, acc = model.evaluate(test_x, test_y)
print('Test loss:', loss)
print('Test accuracy:', acc)

# save model
data = {"max_len": max_len, "vocab_len": vocab_len, "char_index": char_index}
save_model(model, data, 'serving')
#model.save('Filterd_model_LSTM_'+str(acc))


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 15, 256)           318464    
_________________________________________________________________
dropout (Dropout)            (None, 15, 256)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 512)               1574912   
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 2)                 1026      
_________________________________________________________________
activation (Activation)      (None, 2)                 0         
Total params: 1,894,402
Trainable params: 1,894,402
Non-trainable params: 0
______________________________________________

Epoch 47/200
64466/64466 [==============================] - 23s 354us/sample - loss: 0.0175 - accuracy: 0.9942 - val_loss: 0.4329 - val_accuracy: 0.9148
Epoch 48/200
64466/64466 [==============================] - 23s 354us/sample - loss: 0.0166 - accuracy: 0.9942 - val_loss: 0.4439 - val_accuracy: 0.9174
Epoch 49/200
64466/64466 [==============================] - 23s 355us/sample - loss: 0.0145 - accuracy: 0.9957 - val_loss: 0.4670 - val_accuracy: 0.9166
Epoch 50/200
64466/64466 [==============================] - 23s 353us/sample - loss: 0.0150 - accuracy: 0.9953 - val_loss: 0.4396 - val_accuracy: 0.9180
Epoch 51/200
64466/64466 [==============================] - 23s 352us/sample - loss: 0.0125 - accuracy: 0.9960 - val_loss: 0.4616 - val_accuracy: 0.9157
Epoch 52/200
64466/64466 [==============================] - 23s 351us/sample - loss: 0.0140 - accuracy: 0.9954 - val_loss: 0.4646 - val_accuracy: 0.9176
Epoch 53/200
64466/64466 [==============================] - 23s 350us/sample - los

64466/64466 [==============================] - 23s 354us/sample - loss: 0.0049 - accuracy: 0.9986 - val_loss: 0.5542 - val_accuracy: 0.9172
Epoch 101/200
64466/64466 [==============================] - 23s 355us/sample - loss: 0.0080 - accuracy: 0.9977 - val_loss: 0.4801 - val_accuracy: 0.9201
Epoch 102/200
27424/27424 [==============================] - 10s 359us/sample - loss: 0.4962 - accuracy: 0.9190
Test loss: 0.4962320103935507
Test accuracy: 0.91901255
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: serving\server.model\assets


In [10]:
loss, acc = model.evaluate(test_x, test_y)
print('Test loss:', loss)
print('Test accuracy:', acc)



27424/27424 [==============================] - 9s 345us/sample - loss: 0.4962 - accuracy: 0.9190
Test loss: 0.4962320103935507
Test accuracy: 0.91901255


In [11]:
# build model  GRU

callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10)

model = Sequential()
model.add(LSTM(512, return_sequences=True, input_shape=(max_len, vocab_len)))
model.add(Dropout(0.2))
model.add(GRU(2014, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(2))
model.add(Activation('softmax'))
model.summary()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


# train model
model.fit(train_x, train_y, batch_size=128, nb_epoch=200, validation_data=(test_x, test_y), callbacks=[callback])

# test model
loss, acc = model.evaluate(test_x, test_y)
print('Test loss:', loss)
print('Test accuracy:', acc)

# save model
data = {"max_len": max_len, "vocab_len": vocab_len, "char_index": char_index}
#save_model(model, data, 'E://NCS//gender-prediction-master//model//')
save_model(model, data, 'serving_GRU')
#model.save('Filterd_model_GRU_'+str(acc))

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 15, 512)           1161216   
_________________________________________________________________
dropout_2 (Dropout)          (None, 15, 512)           0         
_________________________________________________________________
gru (GRU)                    (None, 2014)              15268134  
_________________________________________________________________
dropout_3 (Dropout)          (None, 2014)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 4030      
_________________________________________________________________
activation_1 (Activation)    (None, 2)                 0         
Total params: 16,433,380
Trainable params: 16,433,380
Non-trainable params: 0
__________________________________________

64466/64466 [==============================] - 59s 922us/sample - loss: 0.0136 - accuracy: 0.9957 - val_loss: 0.4618 - val_accuracy: 0.9201
Epoch 47/200
64466/64466 [==============================] - 59s 922us/sample - loss: 0.0119 - accuracy: 0.9959 - val_loss: 0.4721 - val_accuracy: 0.9197
Epoch 48/200
64466/64466 [==============================] - 59s 921us/sample - loss: 0.0135 - accuracy: 0.9955 - val_loss: 0.4820 - val_accuracy: 0.9174
Epoch 49/200
64466/64466 [==============================] - 59s 922us/sample - loss: 0.0152 - accuracy: 0.9952 - val_loss: 0.4296 - val_accuracy: 0.9189
Epoch 50/200
64466/64466 [==============================] - 59s 922us/sample - loss: 0.0156 - accuracy: 0.9947 - val_loss: 0.4348 - val_accuracy: 0.9184
Epoch 51/200
64466/64466 [==============================] - 59s 923us/sample - loss: 0.0140 - accuracy: 0.9952 - val_loss: 0.4387 - val_accuracy: 0.9194
Epoch 52/200
64466/64466 [==============================] - 60s 925us/sample - loss: 0.0120 - a

In [12]:
loss, acc = model.evaluate(test_x, test_y)
print('Test loss:', loss)
print('Test accuracy:', acc)


27424/27424 [==============================] - 17s 608us/sample - loss: 0.5022 - accuracy: 0.9197
Test loss: 0.5021752181319434
Test accuracy: 0.9197054


In [13]:
# build model Bidiorectional LSTM

callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10)

model = Sequential()
#model.add(Bidirectional(LSTM(512, return_sequences=True, input_shape=(max_len, vocab_len))))
model.add(Bidirectional(LSTM(512, return_sequences=True), input_shape=(max_len, vocab_len)))
model.add(Dropout(0.2))
model.add(LSTM(512, return_sequences=False))
model.add(Dropout(0.1))
model.add(Dense(2))
model.add(Dropout(0.2))
model.add(Dense(2))
model.add(Activation('softmax'))
model.summary()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# train model
model.fit(train_x, train_y, batch_size=128, epochs=200, validation_data=(test_x, test_y), callbacks=[callback])

# test model
loss, acc = model.evaluate(test_x, test_y)
print('Test loss:', loss)
print('Test accuracy:', acc)

# save model
data = {"max_len": max_len, "vocab_len": vocab_len, "char_index": char_index}
#save_model(model, data, 'E://NCS//gender-prediction-master//model//')
save_model(model, data, 'serving_BiLSTM')
#model.save('Filterd_model_BiLSTM_'+str(acc))

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional (Bidirectional (None, 15, 1024)          2322432   
_________________________________________________________________
dropout_4 (Dropout)          (None, 15, 1024)          0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 512)               3147776   
_________________________________________________________________
dropout_5 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 1026      
_________________________________________________________________
dropout_6 (Dropout)          (None, 2)                 0         
_________________________________________________________________
dense_3 (Dense)              (None, 2)                

64466/64466 [==============================] - 41s 634us/sample - loss: 0.0410 - accuracy: 0.9806 - val_loss: 0.5115 - val_accuracy: 0.9206
Epoch 46/200
64466/64466 [==============================] - 41s 639us/sample - loss: 0.0419 - accuracy: 0.9804 - val_loss: 0.5363 - val_accuracy: 0.9213
Epoch 47/200
64466/64466 [==============================] - 41s 635us/sample - loss: 0.0401 - accuracy: 0.9821 - val_loss: 0.5069 - val_accuracy: 0.9219
Epoch 48/200
64466/64466 [==============================] - 41s 634us/sample - loss: 0.0419 - accuracy: 0.9810 - val_loss: 0.4536 - val_accuracy: 0.9207
Epoch 49/200
64466/64466 [==============================] - 41s 632us/sample - loss: 0.0420 - accuracy: 0.9807 - val_loss: 0.4308 - val_accuracy: 0.9191
Epoch 50/200
64466/64466 [==============================] - 41s 637us/sample - loss: 0.0392 - accuracy: 0.9814 - val_loss: 0.4704 - val_accuracy: 0.9208
Epoch 51/200
64466/64466 [==============================] - 41s 636us/sample - loss: 0.0379 - a

In [14]:
loss, acc = model.evaluate(test_x, test_y)
print('Test loss:', loss)
print('Test accuracy:', acc)


27424/27424 [==============================] - 16s 600us/sample - loss: 0.5443 - accuracy: 0.9243
Test loss: 0.5442555474716584
Test accuracy: 0.9243364


In [16]:
# model2=load_model('mymodel_epoch50')
# loss, acc = model2.evaluate(test_x, test_y)
# print('Test loss:', loss)
# print('Test accuracy:', acc)


In [ ]:
print(data)

In [1]:
import os
import json
import numpy as np
import tensorflow as tf



#from keras.models import load_model
from tensorflow.keras.models import Sequential, load_model

import tensorflow as tf

class GenderAPI:
    def __init__(self):
        #self.graph = tf.compat.v1.get_default_graph() #tf.get_default_graph()
        #self.model=load_model('Filterd_model_BiLSTM_0.9210641')
        data={'max_len': 15, 'vocab_len': 54, 'char_index': {'H': 0, 'd': 1, 'a': 2, 'C': 3, 'K': 4, 's': 5, 'Z': 6, 'z': 7, 'P': 8, 'N': 9, 't': 10, 'R': 11, 'b': 12, 'L': 13, 'B': 14, 'n': 15, 'S': 16, 'x': 17, 'Y': 18, 'U': 19, 'END': 20, 'r': 21, 'F': 22, 'e': 23, 'm': 24, 'v': 25, 'W': 26, 'f': 27, 'o': 28, 'h': 29, 'T': 30, 'w': 31, 'j': 32, 'q': 33, 'y': 34, 'i': 35, 'u': 36, 'O': 37, 'c': 38, 'D': 39, 'J': 40, 'V': 41, ' ': 42, 'g': 43, 'I': 44, 'k': 45, 'G': 46, 'p': 47, 'X': 48, 'A': 49, 'Q': 50, 'E': 51, 'l': 52, 'M': 53}}
        self.model, data = self.read_model('serving')
        self.max_len = data['max_len']
        self.vocab_len = data['vocab_len']
        self.char_index = data['char_index']

    def vector(self, i, n):
        tmp = np.zeros(n)
        tmp[i] = 1
        return tmp

    def read_model(self, network_path):
        if not os.path.exists(network_path):
            raise ValueError('Path not found : {}'.format(network_path))
        dat = json.loads(open(os.path.join(network_path, 'data.json')).read())
        #dat=data
        print(dat)
        mod = load_model(os.path.join(network_path, 'server.model'))
        #mod=load_model('Filterd_model_BiLSTM_0.9210641')
        return mod,dat

    def predict(self, names, labelize=True):
        """
        Returns gender of given names

        Args:
            names:      list of strings
            labelize:   returns 'M' or 'F' labels if set to True,
                        returns list of porbabilities otherwise
        """
        #with self.graph.as_default():
            # format input
        #print(names)
        names = [s.lower() for s in names]
        #print("lower",names)

        names = [list(i)+['END']*(self.max_len-len(i)) for i in names]
        names = [[self.vector(self.char_index[j], self.vocab_len) for j in i] for i in names]
        names = np.asarray(names)

        # predict gender
        out = self.model.predict(names).tolist()
        print(out)
        return [('M' if p[0] > p[1] else 'F') for p in out] if labelize else out


In [ ]:
#get array of data from postman post ///// set headers :key,value , set url for POST // inset json body ["Kelvin","maria"]

from flask import Flask, request, jsonify
#from gender_api import GenderAPI

# Init Flask and GenderAPI
app = Flask(__name__)
api = GenderAPI()

@app.route('/')
def hello_world():
    return "Welcome to GenderAPI. Please request on /predict using \
    'Content-Type: application/json' header and a json array of names in the body."

@app.route('/predict', methods=['POST'])
def predict():
    names = request.get_json()
    print(names)
    labels = api.predict(names)
    print(labels)
    return jsonify(labels)

# Run Flask
app.run(host='127.0.0.1', port=5000, debug=False)


In [2]:
#get array of data from HTML form /// example  "Kelvin","maria"



from flask import Flask, request,redirect, url_for,  jsonify,render_template
#from gender_api import GenderAPI

# Init Flask and GenderAPI
app = Flask(__name__)
api = GenderAPI()


@app.route('/')
def my_form():
    return render_template('form.html')

@app.route('/', methods=['POST'])

# def my_form_post():
#     text = request.form['text']
#     processed_text = text.upper()
#     return processed_text

def predict():
    names = request.form['text']
    print(str(names))
    names=str(names).split(",")
    print(names)
    labels = api.predict(names)
    return jsonify(labels)



app.run(host='127.0.0.1', port=5000, debug=False)


{'max_len': 15, 'vocab_len': 54, 'char_index': {'n': 0, 'END': 1, 'C': 2, 'D': 3, 'W': 4, 'V': 5, 'r': 6, 'q': 7, 'X': 8, 'j': 9, 'v': 10, 'T': 11, 'E': 12, 'g': 13, 'Z': 14, 'u': 15, 'i': 16, 'l': 17, 'b': 18, 'G': 19, 'h': 20, 'R': 21, 'S': 22, 'x': 23, 'e': 24, 'Y': 25, 'K': 26, 'w': 27, 'z': 28, 'U': 29, 'c': 30, 'N': 31, 'M': 32, 'L': 33, 'A': 34, 'I': 35, 'm': 36, 'O': 37, 'k': 38, ' ': 39, 'f': 40, 's': 41, 'F': 42, 'B': 43, 'J': 44, 'y': 45, 'P': 46, 'H': 47, 'p': 48, 'o': 49, 'a': 50, 't': 51, 'Q': 52, 'd': 53}}
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [05/Feb/2021 05:59:56] "GET / HTTP/1.1" 200 -


abinaya,kurupara
['abinaya', 'kurupara']


127.0.0.1 - - [05/Feb/2021 05:59:59] "POST / HTTP/1.1" 200 -


[[0.009048513136804104, 0.990951418876648], [0.5194451212882996, 0.48055487871170044]]
